<a href="https://colab.research.google.com/github/felipeoliveiralps/Analise-Sentimento-Ecommerce/blob/master/notebooks/NLP/ModeloLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Carregando dataset**

In [ ]:
import pandas as pd

dataSemStopWords = pd.read_csv('../../datasets/dataSemStopWords.csv', engine='python')


In [ ]:
import spacy

In [ ]:
pip install tensorflow

**Tokenização**

* Antes de alimentar nosso  LSTM, precisamos converter o texto em vetores numéricos de tamanho fixo. O código a seguir executa um pipeline completo: primeiro, um Tokenizer do Keras é usado para construir um vocabulário com as 15.000 palavras mais frequentes e transformar nossas avaliações em sequências de números.

* Em seguida, como as redes neurais exigem entradas de comprimento uniforme, aplicamos o Padding para garantir que todas as sequências tenham exatamente 150 tokens, além de dividir o treino/teste.

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=15000, oov_token="<unk>")

dataSemStopWords['texto_lematizado'] = dataSemStopWords['texto_lematizado'].astype(str)

tokenizer.fit_on_texts(dataSemStopWords['texto_lematizado'])

sequences = tokenizer.texts_to_sequences(dataSemStopWords['texto_lematizado'])

max_length = 150
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

from sklearn.model_selection import train_test_split

X = padded_sequences
y = dataSemStopWords['feedback']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

**Class Weights**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)

pesos = compute_class_weight(class_weight='balanced',
                             classes=classes,
                             y=y_train)

print(f"Classes: {classes}")
print(f"Pesos calculados: {pesos}")

In [ ]:
pip install imbalanced-learn


**Construindo o Modelo LSTM**

* Para esta tarefa de classificação de sentimento, irei construir um modelo de rede neural sequencial. A arquitetura começa com uma camada de Embedding, que é responsável por transformar os números inteiros do nosso vocabulário em vetores densos de 128 dimensões. É nesta camada que o modelo aprenderá as relações de significado entre as palavras.

* O núcleo do modelo é uma camada Bidirectional LSTM com 64 unidades. A LSTM (Long Short-Term Memory) é projetada para entender padrões em sequências, e o fato de ser bidirecional permite que ela analise o texto tanto da esquerda para a direita quanto da direita para a esquerda. Isso fornece um contexto muito mais rico de cada palavra, melhorando a capacidade do modelo de capturar a intenção do texto.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

vocab_size = min(len(tokenizer.word_index) + 1, 15000)
embedding_dim = 128
max_length = 150

# Construir o modelo
model = Sequential([
    Embedding(vocab_size, embedding_dim),

    Bidirectional(LSTM(64)),
    Dropout(0.4),

    Dense(64, activation='relu'),
    Dropout(0.5),

    Dense(3, activation='softmax')
])

**Preparando o dicionário dos pesos**

In [ ]:
class_weight_dict = {i: weight for i, weight in enumerate(pesos)}

**Compilando o modelo com ADAM como optimizador**
* Escolhi ele pois é um algoritmo robusto e amplamente utilizado que ajusta os pesos da rede de forma eficiente para minimizar os erros

In [ ]:

model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)



**Treinamento com Callbacks**
* Com o modelo compilado, iniciarei a fase de treinamento. Para evitar overfitting e garantir que será salvo a versão mais performática do nosso modelo, utilizarei callbacks, que irá monitorar o processo de treino a cada época.

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

model_checkpoint = ModelCheckpoint(
    filepath='melhor_modelo.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=2,
    verbose=1,
    mode='min'
)

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=15,
    batch_size=64,
    class_weight=class_weight_dict,
    callbacks=[early_stopping, model_checkpoint]
)


from tensorflow.keras.models import load_model

print("\nCarregando o melhor modelo salvo...")
model = load_model('melhor_modelo.keras')
print("Melhor modelo carregado com sucesso.")


**Métricas**

In [ ]:
model.summary()

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns


print("\n--- Avaliação Final no Conjunto de Teste ---")

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print(f"Loss no Teste: {loss:.4f}")
print(f"Acurácia no Teste: {accuracy * 100:.2f}%")

y_pred_probs = model.predict(X_test)

_classes = np.argmax(y_pred_probs, axis=1)

# 3. Gerar Relatório de Classificação e Matriz de Confusão
print("\nRelatório de Classificação:")
print(classification_report(y_test, _classes, target_names=['Negativo', 'Neutro', 'Positivo']))

print("\nMatriz de Confusão:")
cm = confusion_matrix(y_test, _classes)

# Visualizar matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=['Negativo', 'Neutro', 'Positivo'],
            yticklabels=['Negativo', 'Neutro', 'Positivo'])
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.show()